In [3]:
import pymongo
from pprint import pprint
import pandas as pd
import numpy as np
from IPython.display import Image

In [4]:
connection =pymongo.MongoClient("localhost",27017)
db = connection.python

for i in db.funds.find():
    pprint(i)

{'_id': ObjectId('61167af198608fe75d51477e'),
 'funding_rounds': [{'raised_amount': 7172702, 'round_code': 'a'},
                    {'raised_amount': 7914665, 'round_code': 'b'},
                    {'raised_amount': 7915102, 'round_code': 'c'},
                    {'raised_amount': 9297130, 'round_code': 'd'},
                    {'raised_amount': 7519725, 'round_code': 'e'},
                    {'raised_amount': 3479603, 'round_code': 'f'}],
 'name': 'Facebook'}
{'_id': ObjectId('61167af198608fe75d51477f'),
 'funding_rounds': [{'raised_amount': 2716902, 'round_code': 'a'},
                    {'raised_amount': 9737827, 'round_code': 'b'},
                    {'raised_amount': 9606464, 'round_code': 'c'},
                    {'raised_amount': 618771, 'round_code': 'd'},
                    {'raised_amount': 509318, 'round_code': 'e'},
                    {'raised_amount': 1924125, 'round_code': 'f'}],
 'name': 'Google'}
{'_id': ObjectId('61167af198608fe75d514780'),
 'funding_rounds':


<br><br>

---

### \$size
- you cannot query size of array directly. 
<br>

#### Solutions

1) You can create different field specifying how many things that you have in an array
2) You can utilize project the size of array through \\$project and use \\$match
3) \$expr
    - \\$expr can build query expressions that compare fields from the same document in a $match stage.


In [5]:
for i in db.funds.aggregate([{"$match":{"funding_rounds" :{"$size":6}}},{"$limit":1}]):
    pprint(i)
    
#but, you can't query like this:
#db.funds.aggregate([{"$match":{"funding_rounds":{"$size":{"$gt":5}}}}])

{'_id': ObjectId('61167af198608fe75d51477e'),
 'funding_rounds': [{'raised_amount': 7172702, 'round_code': 'a'},
                    {'raised_amount': 7914665, 'round_code': 'b'},
                    {'raised_amount': 7915102, 'round_code': 'c'},
                    {'raised_amount': 9297130, 'round_code': 'd'},
                    {'raised_amount': 7519725, 'round_code': 'e'},
                    {'raised_amount': 3479603, 'round_code': 'f'}],
 'name': 'Facebook'}


#### use of project and match combination

In [19]:


for i in db.funds.aggregate([{"$project":{"_id":0,
                                          "name":1,
                                          "num_of_size":{"$size":"$funding_rounds"},
                                         "funding_rounds":1}},
                            {"$match":{"num_of_size":{"$eq":6}}},
                            {"$limit":1}]):
    pprint(i)
    
    
#db.accommodations.find( { $where: "this.name.length > 1" } );

{'funding_rounds': [{'raised_amount': 7172702, 'round_code': 'a'},
                    {'raised_amount': 7914665, 'round_code': 'b'},
                    {'raised_amount': 7915102, 'round_code': 'c'},
                    {'raised_amount': 9297130, 'round_code': 'd'},
                    {'raised_amount': 7519725, 'round_code': 'e'},
                    {'raised_amount': 3479603, 'round_code': 'f'}],
 'name': 'Facebook',
 'num_of_size': 6}


#### use of \$expr

In [22]:
for i in db.funds.aggregate([{"$match":{"$expr":{"$gt":[{"$size":"$funding_rounds"},5]}}},{"$limit":1}]):
    pprint(i)
    
#funding_rounds의 값의 사이즈가 5이상인 것들을 매치되는 대로 리턴해달라는 의미가 된다.

{'_id': ObjectId('61167af198608fe75d51477e'),
 'funding_rounds': [{'raised_amount': 7172702, 'round_code': 'a'},
                    {'raised_amount': 7914665, 'round_code': 'b'},
                    {'raised_amount': 7915102, 'round_code': 'c'},
                    {'raised_amount': 9297130, 'round_code': 'd'},
                    {'raised_amount': 7519725, 'round_code': 'e'},
                    {'raised_amount': 3479603, 'round_code': 'f'}],
 'name': 'Facebook'}


---

### \$slice

In [7]:
for i in db.funds.aggregate([{"$project":{"_id":1,"name":1,"middle_rounds":{"$slice":["$funding_rounds",2,3]}}}]):
    pprint(i)

{'_id': ObjectId('61167af198608fe75d51477e'),
 'middle_rounds': [{'raised_amount': 7915102, 'round_code': 'c'},
                   {'raised_amount': 9297130, 'round_code': 'd'},
                   {'raised_amount': 7519725, 'round_code': 'e'}],
 'name': 'Facebook'}
{'_id': ObjectId('61167af198608fe75d51477f'),
 'middle_rounds': [{'raised_amount': 9606464, 'round_code': 'c'},
                   {'raised_amount': 618771, 'round_code': 'd'},
                   {'raised_amount': 509318, 'round_code': 'e'}],
 'name': 'Google'}
{'_id': ObjectId('61167af198608fe75d514780'),
 'middle_rounds': [{'raised_amount': 5998487, 'round_code': 'c'},
                   {'raised_amount': 6179664, 'round_code': 'd'},
                   {'raised_amount': 2567725, 'round_code': 'e'}],
 'name': 'Naver'}
{'_id': ObjectId('61167af198608fe75d514781'),
 'middle_rounds': [{'raised_amount': 4807965, 'round_code': 'c'},
                   {'raised_amount': 7506110, 'round_code': 'd'},
                   {'raised_amo